In [ ]:
import requests
from tenacity import retry, stop_after_attempt, wait_fixed
import requests_cache

In [3]:
import duckdb
from tqdm import tqdm
conn = duckdb.connect("dados_namus.duckdb")

In [2]:
# Configuração do cache em disco (o arquivo 'namus_cache' armazenará as respostas)
requests_cache.install_cache('namus_cache', expire_after=86400)  # 1 dia de cache

In [3]:
@retry(stop=stop_after_attempt(5), wait=wait_fixed(2))
def make_request(url, payload=None, headers=None, method="POST"):
    try:
        if method == "POST":
            response = requests.post(url, json=payload, headers=headers)
        elif method == "GET":
            response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as e:
        print(f"Erro HTTP {response.status_code}: {response.text}")
        raise

In [4]:
"""
def save_records_to_duckdb(records, conn):
    
    #Função auxiliar para salvar uma lista de registros no DuckDB.

    for record in records:
        conn.execute("INSERT INTO UnidentifiedPersons VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", (
            record.get('idFormatted'),
            record.get('caseNumber'),
            record.get('dateFound'),
            record.get('estimatedAgeFrom'),
            record.get('estimatedAgeTo'),
            record.get('cityOfRecovery'),
            record.get('countyDisplayNameOfRecovery'),
            record.get('stateOfRecovery'),
            record.get('sex'),
            record.get('raceEthnicity'),
            record.get('modifiedDateTime'),
            record.get('namus2Number'),
            record.get('stateDisplayNameOfRecovery')
        ))
"""

'\ndef save_records_to_duckdb(records, conn):\n    \n    #Função auxiliar para salvar uma lista de registros no DuckDB.\n\n    for record in records:\n        conn.execute("INSERT INTO UnidentifiedPersons VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", (\n            record.get(\'idFormatted\'),\n            record.get(\'caseNumber\'),\n            record.get(\'dateFound\'),\n            record.get(\'estimatedAgeFrom\'),\n            record.get(\'estimatedAgeTo\'),\n            record.get(\'cityOfRecovery\'),\n            record.get(\'countyDisplayNameOfRecovery\'),\n            record.get(\'stateOfRecovery\'),\n            record.get(\'sex\'),\n            record.get(\'raceEthnicity\'),\n            record.get(\'modifiedDateTime\'),\n            record.get(\'namus2Number\'),\n            record.get(\'stateDisplayNameOfRecovery\')\n        ))\n'

In [5]:
import json

headers = {
	'Content-Type': 'application/json'
}

def save_records_to_duckdb(records, conn):
    for record in records:
        # Realiza uma segunda requisição via GET para obter dados detalhados de cada caso
        details_url = f'https://www.namus.gov/api/CaseSets/NamUs/MissingPersons/Cases/{record["idFormatted"].replace("MP", "")}'
        details = make_request(details_url, method="GET")
        
        # Verifique se o índice existe antes de acessar listas de detalhes e define None se não existir
        image_href = details.get('images', [{}])[0].get('files', {}).get('original', {}).get('href') if details.get('images') else None
        investigating_agency_name = details.get('investigatingAgencies', [{}])[0].get('name') if details.get('investigatingAgencies') else None
        investigating_case_number = details.get('investigatingAgencies', [{}])[0].get('caseNumber') if details.get('investigatingAgencies') else None
        case_date_reported = details.get('investigatingAgencies', [{}])[0].get('dateReported') if details.get('investigatingAgencies') else None

        vehicles_json = json.dumps(details.get('vehicles', None))

        conn.execute("""
            INSERT INTO MissingPersons VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            details.get('idFormatted', None),
            details.get('createdDateTime', None),
            details.get('modifiedDateTime', None),
            details.get('subjectIdentification', {}).get('firstName', None),
            details.get('subjectIdentification', {}).get('middleName', None),
            details.get('subjectIdentification', {}).get('lastName', None),
            details.get('subjectIdentification', {}).get('computedMissingMinAge', None),
            details.get('subjectIdentification', {}).get('computedMissingMaxAge', None),
            details.get('circumstances', {}).get('circumstancesOfDisappearance', None),
            details.get('subjectDescription', {}).get('sex', {}).get('name', None),
            details.get('physicalDescription', {}).get('hairColor', {}).get('name', None),
            details.get('physicalDescription', {}).get('leftEyeColor', {}).get('name', None),
            details.get('physicalDescription', {}).get('rightEyeColor', {}).get('name', None),
            details.get('subjectDescription', {}).get('primaryEthnicity', {}).get('name', None),
            details.get('sighting', {}).get('date', None),
            details.get('sighting', {}).get('address', {}).get('city', None),
            details.get('sighting', {}).get('address', {}).get('state', {}).get('displayName', None),
            details.get('sighting', {}).get('address', {}).get('county', {}).get('displayName', None),
            image_href,
            details.get('sighting', {}).get('publicGeolocation', {}).get('coordinates', {}).get('lat', None),
            details.get('sighting', {}).get('publicGeolocation', {}).get('coordinates', {}).get('lon', None),
            details.get('primaryInvestigatingAgency', {}).get('name', None),
            investigating_agency_name,
            investigating_case_number,
            case_date_reported,
            details.get('permissionToPublish', None),
            details.get('viewPermission', None),
            details.get('hrefDefaultImageThumbnail', None),
            details.get('hrefQRCode', None),
            details.get('caseIsResolved', None),
            vehicles_json
        ))

In [7]:
conn.execute("""
    CREATE TABLE IF NOT EXISTS MissingPersons (
        idFormatted TEXT,
        createdDateTime TEXT,
        modifiedDateTime TEXT,
        firstName TEXT,
        middleName TEXT,
        lastName TEXT,
        computedMissingMinAge INTEGER,
        computedMissingMaxAge INTEGER,
        circumstancesOfDisappearance TEXT,
        sex TEXT,
        hairColor TEXT,
        leftEyeColor TEXT,
        rightEyeColor TEXT,
        ethnicity TEXT,
        sightingDate TEXT,
        sightingCity TEXT,
        sightingState TEXT,
        sightingCounty TEXT,
        imageHref TEXT,
        sightingLat FLOAT,
        sightingLon FLOAT,
        primaryAgencyName TEXT,
        investigatingAgencyName TEXT,
        investigatingCaseNumber TEXT,
        caseDateReported TEXT,
        permissionToPublish BOOLEAN,
        viewPermission TEXT,
        hrefThumbnail TEXT,
        hrefQRCode TEXT,
        caseIsResolved BOOLEAN,
        vehicles JSON
    )
""")

In [8]:
take = 100  # Número de registros por requisição
skip = 4300      # Ponto de partida para cada página de resultados
url = 'https://www.namus.gov/api/CaseSets/NamUs/MissingPersons/Search'

# Configuração inicial da requisição
payload = {
    "predicates": [],
    "take": 50,
    "skip": 0,
    "projections": [
        "idFormatted",
        "dateOfLastContact",
        "lastName",
        "firstName",
        "computedMissingMinAge",
        "computedMissingMaxAge",
        "cityOfLastContact",
        "countyDisplayNameOfLastContact",
        "stateDisplayNameOfLastContact",
        "gender",
        "raceEthnicity",
        "modifiedDateTime",
        "namus2Number"
    ],
    "orderSpecifications": [
        {
            "field": "dateOfLastContact",
            "direction": "Descending"
        }
    ],
    "documentFragments": [
        "birthDate"
    ]
}

In [9]:
# Primeira requisição para obter o total de registros
data = make_request(url, payload, headers)

total_count = 10000 #data['count']
save_records_to_duckdb(data['results'], conn)
skip += take

with tqdm(total=total_count, initial=skip, desc="Baixando registros") as pbar:
	while skip < total_count:
		if skip + take > total_count:
			take = total_count - skip  # Ajusta `take` para baixar apenas o necessário
                
		payload['skip'] = skip
		payload['take'] = take
	
		data = make_request(url, payload, headers)
		
		save_records_to_duckdb(data['results'], conn)
		pbar.update(len(data['results']))
		skip += take

Baixando registros: 100%|██████████| 10000/10000 [1:55:50<00:00,  1.24s/it]


In [10]:
conn.close()